In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

2024-04-03 12:45:15.924998: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-03 12:45:15.948725: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-03 12:45:16.401882: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


cpu


In [2]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

# Read data from CSV files
notes_df = pd.read_csv('/home/admin1/Desktop/LSTM/csv_dataset/notes.csv')
test_df = pd.read_csv('/home/admin1/Desktop/LSTM/csv_dataset/testset.csv')

# Extract data and labels from the test set
data_test = test_df[['x_test', 'future']].to_numpy()
x_test_string = data_test[:, 0]
y_test_string = data_test[:, 1]

# Convert string representations to lists
x_test = []
y_test = []

for i in x_test_string:
    # Remove square brackets and newline characters
    i = i.strip('[]\n')
    # Split the string and convert elements to integers
    input_x_test = [int(j) for j in i.split()]
    x_test.append(input_x_test)

for i in y_test_string:
    # Remove square brackets and newline characters
    i = i.strip('[]\n')
    # Split the string and convert elements to integers
    input_y_test = [int(j) for j in i.split()]
    y_test.append(input_y_test)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Create a dictionary of unique notes
notes_ = notes_df.iloc[:, 0].to_numpy()
unique_notes = dict(enumerate(notes_.flatten(), 0))

In [3]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
class MusicDataset(Dataset):
    def __init__(self, data , data_labels):
        self.data = data
        self.data_labels = data_labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx] , self.data_labels[idx]


test_set = MusicDataset(x_test,y_test)
testloader = torch.utils.data.DataLoader(test_set, batch_size=1,
                                         shuffle=False, num_workers=2)

In [4]:
Net = torch.load('/home/admin1/Desktop/LSTM/state_dictionary/lstmmodel_basic.pth')
print(Net)

OrderedDict([('embedding.weight', tensor([[ 0.7544,  0.3040,  1.6204,  ...,  1.0693,  0.3593,  1.7222],
        [-0.6374,  2.1065,  0.2349,  ...,  0.1555, -1.1250, -1.3722],
        [ 0.9668,  0.8912,  0.6208,  ..., -0.1869, -0.0127, -1.1430],
        ...,
        [ 0.3607, -0.6401, -0.3260,  ..., -0.2234, -1.3026,  0.3360],
        [-0.5505,  0.0945,  0.1333,  ..., -1.0991,  1.2944, -0.4325],
        [-0.8061,  1.4256, -1.8271,  ..., -1.4185, -0.7938, -0.5755]])), ('lstm.weight_ih_l0', tensor([[-0.5117, -0.2418,  0.2535,  ...,  0.0728, -0.6606, -0.0609],
        [ 0.1960,  0.2230, -0.2698,  ...,  0.3531,  0.2382, -0.2042],
        [-0.0205,  0.1092, -0.1290,  ...,  0.1861,  0.2731,  0.3246],
        ...,
        [-0.3047, -0.0027, -0.0142,  ..., -0.3275, -0.1612,  0.4778],
        [ 0.2551,  0.1787, -0.3990,  ..., -0.2909,  0.3369, -0.6293],
        [ 0.2171, -0.1421,  0.3830,  ..., -0.2837, -0.5508,  0.4782]])), ('lstm.weight_hh_l0', tensor([[ 0.1808, -0.0802, -0.3100,  ...,  0.3902,

In [5]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, num_classes):
        super(LSTM, self).__init__()
        self.num_classes = num_classes
        
        self.embedding = nn.Embedding(num_classes, 100)
        self.lstm = nn.LSTM(input_size=100, hidden_size=256, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]  # Take the last timestep's output
        x = self.fc1(lstm_out)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

In [6]:
import torch
import numpy as np
from torch.utils.data import DataLoader

num_classes = 182
seq_length = 32

model = LSTM(num_classes)  # Instantiate your model
model.load_state_dict(torch.load('/home/admin1/Desktop/LSTM/state_dictionary/lstmmodel_basic.pth'))  # Load the state dictionary

# Define testloader
testloader = DataLoader(test_set, batch_size=1, shuffle=False, num_workers=2)

total_preds = 0
correct_preds = 0
future_preds = 8
for i, data in enumerate(testloader, 0):

    input , label = data
    cumm_output = torch.zeros(0,len(unique_notes)).to(device)
    cumm_label  = np.array([],dtype=int)
    for k in range(future_preds):
        output = model(input.to(device))
        cumm_output = torch.cat((cumm_output,output))
        cumm_label = np.concatenate((cumm_label,label[:,k]))
        next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
        total_preds += input.shape[0]
        correct_preds += torch.sum(torch.argmax(output,1) == label[:,k].to(device))
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
                                               for ind,j in enumerate(input)])[:,1:]) 
test_acc =  float(correct_preds)/float(total_preds) *100 
testreport ="Testing Accuracy : \n correct predictions  : {} \n total predictions : {} \n Testing Accuracy : {} \n ------------------------\n".format(correct_preds,total_preds,test_acc)
print(testreport)  

Testing Accuracy : 
 correct predictions  : 9916 
 total predictions : 62712 
 Testing Accuracy : 15.81196581196581 
 ------------------------



In [7]:
#Working mostly better than the other one, hear the music0.mid file, it is not that bad

In [8]:
def note_to_int(note):
    # Dictionary mapping note names to MIDI numbers
    note_dict = {
        'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3, 'E': 4, 'F': 5,
        'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8, 'Ab': 8, 'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
    }
    if isinstance(note, str):  # if note is a string
        if note.isdigit():  # if note is a number string
            return int(note)
        else:  # if note is a note string
            # Separate the pitch and the octave (e.g. 'E5' -> ('E', '5'))
            pitch, octave = note[:-1], note[-1]
            # Replace '-' with 'b' in the pitch
            pitch = pitch.replace('-', 'b')
            # Calculate the MIDI number
            midi_num = note_dict[pitch] + (int(octave) + 1) * 12
            return midi_num
    elif isinstance(note, int):  # if note is an integer
        return note
    else:
        raise ValueError(f"Invalid note: {note}")

In [9]:
import mido

def convert_to_midi(prediction_output, path):
    mid = mido.MidiFile()
    track = mido.MidiTrack()
    mid.tracks.append(track)

    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            for current_note in notes_in_chord:
                cn = note_to_int(current_note)
                # Create note_on and note_off events
                track.append(mido.Message('note_on', note=cn, velocity=64, time=0))
                track.append(mido.Message('note_off', note=cn, velocity=64, time=480))  # Assuming note length of 480 ticks
        # pattern is a note
        else:
            cn = note_to_int(pattern)
            # Create note_on and note_off events
            track.append(mido.Message('note_on', note=cn, velocity=64, time=0))
            track.append(mido.Message('note_off', note=cn, velocity=64, time=480))  # Assuming note length of 480 ticks

    mid.save(path)
    print(f"MIDI file saved at: {path}")

In [10]:
import os
import random
import pygame
import numpy as np
import torch


# Initialize pygame
pygame.init()

for j in range(10):
    index = random.randint(0, len(x_test))
    print("Taking the seed tune as follows:")
    print(x_test[index])
    tune = x_test[index]
    input = np.empty((1, 32), dtype=int)
    input[0] = tune
    input = torch.from_numpy(input)
    next_preds = 64
    for i in range(next_preds):
        output = model(input.to(device))
        next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
        input = input.cpu().detach().numpy()
        input = torch.from_numpy(np.array([np.append(j, next_preds[ind])
                                           for ind, j in enumerate(input)])[:, 1:])

        tune = np.insert(tune, -1, next_preds[0])
    tune = [unique_notes[i] for i in tune]
    path = '/home/admin1/Desktop/LSTM/Output2/music' + str(j) + '.midi'

    # Check if the directory exists, if not, create it
    directory = os.path.dirname(path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Try to write the MIDI file
    try:
        convert_to_midi(tune, path)
        print(f"MIDI file saved at: {path}")

        # Load and play the MIDI file
        pygame.mixer.music.load(path)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            pygame.time.Clock().tick(10)  # Adjust playback speed if needed

    except Exception as e:
        print(f"Failed to write MIDI file: {e}")


pygame 2.5.2 (SDL 2.28.2, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
Taking the seed tune as follows:
[162  47  11 162 160  11  76  97  99  17  99  97  50  92  46  50  76  97
  99  17  99  65  97  65  50  65  92  50 159  97  99  17]
MIDI file saved at: /home/admin1/Desktop/LSTM/Output2/music0.midi
MIDI file saved at: /home/admin1/Desktop/LSTM/Output2/music0.midi
Failed to write MIDI file: Couldn't open timidity.cfg
Taking the seed tune as follows:
[ 17 145  42 145  92 145  42  46 145  56  17 145  42 145 116  37 145  42
 145 120  89 170   8  50 170  58  89 170 159  50 170  39]
MIDI file saved at: /home/admin1/Desktop/LSTM/Output2/music1.midi
MIDI file saved at: /home/admin1/Desktop/LSTM/Output2/music1.midi
Failed to write MIDI file: Couldn't open timidity.cfg
Taking the seed tune as follows:
[ 18  58  44 100  58 165 124  58 123  47  15  89  58  37  17  58  11  42
  58  50 170  71 171 172   8  50 170 171 172 165  50 170]
MIDI file saved at: /ho

In [11]:
# import os
# import random

# for j in range(10):
#     index = random.randint(0,len(x_test))
#     print("Taking the seed tune as follows:")
#     print(x_test[index])
#     tune = x_test[index]
#     input = np.empty((1,32),dtype=int)
#     input[0] = tune
#     input = torch.from_numpy(input)
#     next_preds = 64
#     for i in range(next_preds):
#         #output = Net(input.to(device),input.shape[0])
#         output = model(input.to(device))
#         next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
#         input = input.cpu().detach().numpy()
#         input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
#                                                    for ind,j in enumerate(input)])[:,1:]) 

#         tune = np.insert(tune,-1,next_preds[0])
#     tune = [unique_notes[i] for i in tune]
#     path = '/home/admin1/Desktop/LSTM/Output1/music'+str(j)+'.midi'
    
#     # Check if the directory exists, if not, create it
#     directory = os.path.dirname(path)
#     if not os.path.exists(directory):
#         osimport os
# import random

# for j in range(10):
#     index = random.randint(0,len(x_test))
#     print("Taking the seed tune as follows:")
#     print(x_test[index])
#     tune = x_test[index]
#     input = np.empty((1,32),dtype=int)
#     input[0] = tune
#     input = torch.from_numpy(input)
#     next_preds = 64
#     for i in range(next_preds):
#         #output = Net(input.to(device),input.shape[0])
#         output = model(input.to(device))
#         next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
#         iimport os
# import random

# for j in range(10):
#     index = random.randint(0,len(x_test))
#     print("Taking the seed tune as follows:")
#     print(x_test[index])
#     tune = x_test[index]
#     input = np.empty((1,32),dtype=int)
#     input[0] = tune
#     input = torch.from_numpy(input)
#     next_preds = 64
#     for i in range(next_preds):
#         #output = Net(input.to(device),input.shape[0])
#         output = model(input.to(device))
#         next_preds = np.argmax(output.cpu().detach().numpy(),axis=1)
#         input = input.cpu().detach().numpy()
#         input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
#                                                    for ind,j in enumerate(input)])[:,1:]) 

#         tune = np.insert(tune,-1,next_preds[0])
#     tune = [unique_notes[i] for i in tune]
#     path = '/home/admin1/Desktop/LSTM/Output1/music'+str(j)+'.midi'
    
#     # Check if the directory exists, if not, create it
#     directory = os.path.dirname(path)
#     if not os.path.exists(directory):
#         os.makedirs(directory)
    
#     # Try to write the MIDI file
#     try:
#         convert_to_midi(tune, path)
#         print(f"MIDI file saved at: {path}")
#     except Exception as e:
#         print(f"Failed to write MIDI file: {e}")nput = input.cpu().detach().numpy()
#         input = torch.from_numpy(np.array([np.append(j,next_preds[ind]) 
#                                                    for ind,j in enumerate(input)])[:,1:]) 

#         tune = np.insert(tune,-1,next_preds[0])
#     tune = [unique_notes[i] for i in tune]
#     path = '/home/admin1/Desktop/LSTM/Output1/music'+str(j)+'.midi'
    
#     # Check if the directory exists, if not, create it
#     directory = os.path.dirname(path)
#     if not os.path.exists(directory):
#         os.makedirs(directory)
    
#     # Try to write the MIDI file
#     try:
#         convert_to_midi(tune, path)
#         print(f"MIDI file saved at: {path}")
#     except Exception as e:
#         print(f"Failed to write MIDI file: {e}").makedirs(directory)
    
#     # Try to write the MIDI file
#     try:
#         convert_to_midi(tune, path)
#         print(f"MIDI file saved at: {path}")
#     except Exception as e:
#         print(f"Failed to write MIDI file: {e}")